### Packages and funtions

In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
os.getcwd()

'/fast/work/groups/ag_kircher/MisLa/MisLaProject'

In [3]:
import pandas as pd
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
import sys
import numpy as np
import os

sys.path.insert(0, 'utils/')


#conda install -c conda-forge jupyter-resource-usage
from utils import *

import numpy as np
np.seterr(divide='ignore', invalid='ignore')
np.set_printoptions(suppress=True)
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
def blockPrint():
    sys.stdout = open(os.devnull, 'w')
def enablePrint():
    sys.stdout = sys.__stdout__

    
#del sys.modules['XGBoost']
import inspect as i
import sys
#sys.stdout.write(i.getsource(applyModel))

### Funtions

In [ ]:
def prepareData(df):
    
    df = df[~df.index.isna()]
    df = df.fillna(0)
    ## delete all columns with strings
    ind = df.dtypes[(df.dtypes=='int64')|(df.dtypes=='float64')].index
    return df[ind]   


def getLabels(df):
    LabelOld = df['Labels'].str[0]
    LabelOld.name = 'LabelOld'
    LabelNew = df['Labels'].str[-1]
    LabelNew.name = 'LabelNew'
    return [LabelOld,LabelNew]

def deleteConflicting(df):
    df = df[~df['LabelOld'].str.contains('Conflicting',na = False)]
    df = df[~df['LabelNew'].str.contains('Conflicting',na = False)]
    return df

In [ ]:
def applyModel(dataset):
    import pandas
    import matplotlib.pyplot as plt
    from sklearn import model_selection
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC

    # prepare configuration for cross validation test harness
    seed = 7
    # prepare models
    models = []
    #models.append(('LR', LogisticRegression()))
    models.append(('LDA', LinearDiscriminantAnalysis()))
    #models.append(('KNN', KNeighborsClassifier()))
    #models.append(('CART', DecisionTreeClassifier()))
    #models.append(('NB', GaussianNB()))
    #models.append(('SVM', SVC()))

    dataset = dataset[~dataset.index.isna()]
    ind = dataset.dtypes[(dataset.dtypes=='int64')|(dataset.dtypes=='float64')].index
    dataset = dataset[ind]
    dataset = dataset.fillna(0)

    X = dataset.values
    Y = dataset.index
    results = []
    names = []
    scoring = 'accuracy'
    for name, model in models:
        kfold = model_selection.KFold(n_splits=3, random_state=seed,shuffle=True)
        cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        #print(msg)
    return round(cv_results.mean(), 4)


    
from sklearn.utils import resample

def balance_data(dataset, x, up = True):

    temp = dataset[x].value_counts()
    sets = temp.index
    df_resampled = pd.DataFrame()
    for m in sets:
        #  smote = SMOTE(sampling_strategy='minority')
      #  X_sm, y_sm = smote.fit_resample(X, y)

        df = dataset[dataset[x]==m]
        if up ==True:
            n = temp.max()
        else:
            n =temp.min()
            
        df_temp = resample(df, replace=True,    
                          n_samples=n, random_state=123) 
        df_resampled = df_resampled.append(df_temp)

    return df_resampled#.reset_index(drop = True)


def getAcc(data, label, cat, balance,up):
    
    ind = data[label].replace(cs)
    data.index = ind
    data = data[data.index.isin(cat)]
    #l = data[label].value_counts(normalize = True).round(3).to_dict() #(list(map(cs_inv.get, cat)
    
    if balance == True:
        data = balance_data(data, label,up)
    
    ratio = data[label].value_counts().to_dict()

    data = prepareData(data).iloc[:,7:]
    #data = data.drop(columns = ['LabelNew','LabelOld'])   
    print('Cathegories: %s - Label: %s - Dataset size: %s'  % (ratio, label, len(data)))
    acc = applyModel(data)
    print('CV accuracy: ', acc)
    return acc, ratio


### Text

### Get labels

In [7]:
## get the annotated variants
### THIS FILE IS MISSING
file = 'data/GRCh38/variants/annotation/SNV.anno.tsv.gz'

df = pd.read_csv(file, sep = '\t', low_memory=False)
df.index = df['#Chrom'].astype(str) + '-' + df['Pos'].astype(str) +'-' \
            + df['Ref'] +'-' + df['Alt']

## get the dataset created by the ClinVar notebook 
##  for the future - save all info from this set in the ID column in the VCF file that 
## will be annotated
alle = pd.read_csv('alle.csv', sep = '\t',index_col = 'Extra', low_memory=False)
f = alle.iloc[:,1::2]
f = f.T
# unique keeps the order of the labels occurance
aa = f.apply(lambda x: x.dropna().unique())

aa.name = 'Labels'
ff =  df.join(aa,how = 'inner')
# delete duplicates (keep those with max ConsScore)
ff = ff.reset_index()
ff.index = list(range(len(ff)))
gr = ff.groupby(ff['index'])['ConsScore'].idxmax()
joined = ff.loc[gr,:]

joined = joined.set_index('index')


FileNotFoundError: [Errno 2] No such file or directory: 'data/GRCh38/PU/variants/annotation/SNV.anno.tsv.gz'

### Create final dataset 

In [4]:
ID = 'ID'
df21_raw = pd.read_csv('dataRaw/variant_summary.txt.gz',sep = '\t',low_memory=False)
df21 = set_index(df21_raw, ID)
df21['Extra'] = df21['Chromosome'] + '-' + df21['Start'] +'-' + df21['ReferenceAlleleVCF'] +'-' + df21['AlternateAlleleVCF']
df21 = df21.set_index('Extra')




In [5]:
### ALTERNATIVE SINCE A FILE WITH CADD IS MISSING
### Need for calculate how many chanched more than 1
alle = pd.read_csv('dataProduced/alle.csv', sep = '\t',index_col = 'Extra', low_memory=False)
f = alle.iloc[:,1::2]
f = f.T
# unique keeps the order of the labels occurance
aa = f.apply(lambda x: x.dropna().unique())
a= aa[aa.str.len()>0]
df = df21.join(pd.DataFrame(aa), rsuffix = '_df21', lsuffix = '_joined', how = 'inner' )
# get the first and the last labels for a variant, ignore intermediate (there are some)
# with more than 3 labels
df['LabelOld'] = df[0].str[0]
df['LabelNew'] = df[0].str[-1]
labels = ['LabelOld', 'LabelNew']


In [9]:
df.shape

(883702, 40)

In [8]:
pd.crosstab(df['LabelOld'],df['LabelNew'] , margins = True)

LabelNew,Benign,Conflicting interpretations,Pathogenic,Uncertain significance,All
LabelOld,,,,,
Benign,264495,15625,211,1374,281705
Conflicting interpretations,239,2160,1017,149,3565
Pathogenic,531,4409,164157,2891,171988
Uncertain significance,10994,23739,2748,340231,377712
All,276259,45933,168133,344645,834970


In [6]:

pd.crosstab(aa.str[-1],aa.str[0], margins = True)

col_0,Benign,Conflicting interpretations,Pathogenic,Uncertain significance,All
row_0,,,,,
Benign,257393,222,522,10949,269086
Conflicting interpretations,15615,2085,4389,23566,45655
Pathogenic,65,97,137005,2081,139248
Uncertain significance,1282,88,2779,336016,340165
All,274355,2492,144695,372612,794154


In [5]:
df = df21.join(joined, rsuffix = '_df21', lsuffix = '_joined', how = 'inner' )
# get the first and the last labels for a variant, ignore intermediate (there are some)
# with more than 3 labels
df['LabelOld'] = df['Labels'].str[0]
df['LabelNew'] = df['Labels'].str[-1]
labels = ['LabelOld', 'LabelNew']


NameError: name 'joined' is not defined

In [ ]:
#df.to_csv('ClinVarAnnoLabels.csv', sep = '\t')
df = pd.read_csv('dataProduced/ClinVarAnnoLabels.csv', sep = '\t',index_col = 'Unnamed: 0')


/data/gpfs-1/users/nazaretl_c/work/miniconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (19,38,51,73,158) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cs = {'Uncertain significance': 2,
        'Benign':0,
        'Pathogenic':1,
        'Conflicting interpretations':3}
cs_inv = {v: k for k, v in cs.items()}
#.replace(cs)

In [ ]:
# dont run, only for finding correlated features and labels 
#df['LabelOld'] = df['LabelOld'].replace(cs)
#df['LabelNew'] = df['LabelNew'].replace(cs)

#cmatrix = df.corr()
#cmatrix['LabelOld'].sort_values().head(200)
#cmatrix['LabelNew'].sort_values().head(200)

In [ ]:
# drop ClinSigSimple and features that are highly correlated with the labels (if needed)
#df = df.drop(columns = ['ClinSigSimple','NumberSubmitters','dbscSNV-ada_score','ConsScore'])
df = df.drop(columns = ['ConsScore','ClinSigSimple','NumberSubmitters','VariationID','dbscSNV-ada_score'])




In [ ]:
#df = deleteConflicting(df)
df = df[~df['LabelNew'].isna()]
# add an independent ID 
df['ID'] = list(range(len(df)))


# Use changes in Conflicting interpretations and Uncertain significance to test the models?
pd.crosstab(df['LabelOld'], df['LabelNew'])

In [ ]:
LabelNew = df['LabelNew'].replace(cs)
LabelOld = df['LabelOld'].replace(cs)

In [ ]:

#df.index = df['LabelNew'].replace(cs)
df = prepareData(df)
df = df.iloc[:,11:]


In [ ]:
df['LabelNew'] = LabelNew
df['LabelOld'] = LabelOld
df.index = list(range(len(df)))

df.head()

In [ ]:
#df[df['LabelNew']!=df['LabelOld']]

In [ ]:
df.to_csv('dataProduced/ClinVarAnnoLabelsCleanBoth.csv', sep = '\t')

In [ ]:
#noise  = pd.crosstab(df['LabelOld'], df['LabelNew'],normalize = 'columns').values
#noise = noise.round(3)

### 

In [ ]:
#feature = 'Labels' #SubmitterCategories
labels = ['LabelOld', 'LabelNew']

clinicalSig = [[0,1],[0,1,2],[0,1,3],[0,1,2,3]]
compareAll = pd.DataFrame(pd.Series(clinicalSig, name = 'clinicalSig'))
accuracies = []
labels = ['LabelNew']
clinicalSig = [[0,1,2]]


#categories = [[0,1],[0,1,2]]

for sig in clinicalSig:
    for label in labels: # here only new labels
        b = True
        acc, ratio = getAcc(df,label,sig, balance = b,up = False)
        accuracies.append([sig,label,acc])


        


### Run model(s) on the entire dataset 

In [ ]:
#feature = 'Labels' #SubmitterCategories
labels = ['LabelOld', 'LabelNew']
clinicalSig = [[0,1],[0,1,2],[0,1,3],[0,1,2,3]]
compareAll = pd.DataFrame(pd.Series(clinicalSig, name = 'clinicalSig'))
accuracies = []
#labels = ['LabelNew']

#categories = [[0,1],[0,1,2]]

for sig in clinicalSig:
    for label in labels: # here only new labels
        print('Not balanced --------------------')
        b = False
        acc, ratio = getAcc(df,label,sig, balance = b,up = False)
        accuracies.append([sig,label,acc])
        print('Balanced -------------------------')
        b = True
        acc, ratio = getAcc(df,label,sig, balance = b, up = False)
        accuracies.append([sig,label,acc])



In [ ]:
accDF = pd.DataFrame(accuracies)
accDF

### Filter by Gene, ReviewStatus and Consequence with balanced and unbalanaced data

In [ ]:
to_replace = {'reviewed by expert panel' : 3,
           'practice guideline' : 4,
           'criteria provided, multiple submitters, no conflicts':2,
              'criteria provided, single submitter':1,
           'criteria provided, conflicting interpretations': 0,
           'no assertion criteria provided':0
             }
df['ReviewStatus'] = df['ReviewStatus'].replace(to_replace)    

In [ ]:
features = {'GeneID_joined': 2400,
            'SubmitterCategories' :0,
            'Consequence' : 150
           }



In [ ]:
blockPrint()
means = []
stds = []
compare = pd.DataFrame(pd.Series(clinicalSig, name = 'clinicalSig'))

for key in features.keys():
    feature = key
    lim = features[feature]
    balance = True
    a = df[feature].value_counts()
    categories = a[(a>lim) | a.isin([-1,'-1'])].index
    accuracies = pd.DataFrame(pd.Series(categories, name = feature))
  #  ratios = []
    for sig in clinicalSig:
        li = []
        ratios = []

        for cat in categories:
            dataset = df[df[feature] == cat]
            #print(len(dataset))
            acc, ratio = getAcc(dataset, label ,sig, balance = balance)
            li.append(acc)
            #print(li)
            ratios.append(ratio)
        name = str(list(map(cs_inv.get, sig)))    
        accuracies[name] = li #map(float, li)
        accuracies[name + ' RATIO'] = ratios
        print('---------')

    accuracies = accuracies.set_index(feature, drop = True)

    std =pd.Series(accuracies.std(axis = 0), name = 'Std '+feature)
    mean = pd.Series(accuracies.mean(axis = 0), name = 'Mean '+feature)
    
    compare[mean.name] = mean.to_list()
    compare[std.name] = std.to_list()
    
    accuracies = accuracies.append(mean)
    accuracies = accuracies.append(std)
    accuracies.columns = accuracies.columns.str.replace('\[|\]','').str.replace('\'','')
    accuracies.to_csv('tables/'+feature+'.csv', sep = '\t')
    enablePrint()
compare